## Voice Style Control Demo

In [16]:
import os
import torch
from openvoice import se_extractor
from openvoice.api import BaseSpeakerTTS, ToneColorConverter

### Initialization

In [17]:
ckpt_base = 'checkpoints/base_speakers/EN'
ckpt_converter = 'checkpoints/converter'
device="cuda:0" if torch.cuda.is_available() else "cpu"
output_dir = 'outputs'

base_speaker_tts = BaseSpeakerTTS(f'{ckpt_base}/config.json', device=device)
base_speaker_tts.load_ckpt(f'{ckpt_base}/checkpoint.pth')

tone_color_converter = ToneColorConverter(f'{ckpt_converter}/config.json', device=device)
tone_color_converter.load_ckpt(f'{ckpt_converter}/checkpoint.pth')

os.makedirs(output_dir, exist_ok=True)

Loaded checkpoint 'checkpoints/base_speakers/EN/checkpoint.pth'
missing/unexpected keys: [] []
Loaded checkpoint 'checkpoints/converter/checkpoint.pth'
missing/unexpected keys: [] []


### Obtain Tone Color Embedding

The `source_se` is the tone color embedding of the base speaker. 
It is an average of multiple sentences generated by the base speaker. We directly provide the result here but
the readers feel free to extract `source_se` by themselves.

In [18]:
source_se = torch.load(f'{ckpt_base}/en_default_se.pth').to(device)

The `reference_speaker.mp3` below points to the short audio clip of the reference whose voice we want to clone. We provide an example here. If you use your own reference speakers, please **make sure each speaker has a unique filename.** The `se_extractor` will save the `targeted_se` using the filename of the audio and **will not automatically overwrite.**

In [19]:
reference_speaker = 'inputAudio\Edited2.wav'
target_se, audio_name = se_extractor.get_se(reference_speaker, tone_color_converter, target_dir='processed', vad=True)

### Inference

In [20]:




save_path = f'{output_dir}/output_en_default.wav'

# Run the base speaker tts
text = """In July 1992, industrialist Henry Rowan and his wife Betty donated 100 million dollars to the institution, then the largest gift ever to a public college or university. Later that year, the school changed its name to Rowan College of New Jersey to recognize its benefactors’ generosity. The Rowans’ only request was that a College of Engineering be created with a curriculum that would address the shortcomings of engineering education."""
src_path = f'{output_dir}/tmp.wav'
base_speaker_tts.tts(text, src_path, speaker='default', language='English', speed=1.0)

# Run the tone color converter
encode_message = "@MyShell"
tone_color_converter.convert(
    audio_src_path=src_path, 
    src_se=source_se, 
    tgt_se=target_se, 
    output_path=save_path,
    message=encode_message)

 > Text splitted to sentences.
In July 1992, industrialist Henry Rowan and his wife Betty donated 100 million dollars to the institution,
then the largest gift ever to a public college or university.
Later that year, the school changed its name to Rowan College of New Jersey to recognize its benefactors' generosity.
The Rowans' only request was that a College of Engineering be created with a curriculum that would address the shortcomings of engineering education.
 > ===========================
ɪn ˌdʒuˈlaɪ ˈnaɪnˈtin ninety-two*, ˌɪnˈdəstɹiəlɪst ˈhɛnɹi ɹoʊən ənd hɪz waɪf ˈbɛti ˈdoʊˌneɪtɪd wən ˈhənəɹd ˈmɪljən ˈdɔləɹz tɪ ðə ˌɪnstɪˈtuʃən,
 length:142
 length:142
ðɛn ðə ˈlɑɹdʒəst gɪft ˈɛvəɹ tɪ ə ˈpəblɪk ˈkɑlɪdʒ əɹ ˌjunəˈvəɹsəti.
 length:66
 length:66
ˈleɪtəɹ ðət jɪɹ, ðə skuɫ tʃeɪndʒd ɪts neɪm tɪ ɹoʊən ˈkɑlɪdʒ əv nu ˈdʒəɹzi tɪ ˈɹɛkəgˌnaɪz ɪts ˈbɛnəˌfæktəɹz' ˌdʒɛnəɹˈɑsəti.
 length:122
 length:121
ðə ɹowans*' ˈoʊnli ɹɪkˈwɛst wɑz ðət ə ˈkɑlɪdʒ əv ˈɛndʒəˈnɪɹɪŋ bi kɹiˈeɪtɪd wɪθ ə kəɹˈɪkjələm ðət w

**Try with different styles and speed.** The style can be controlled by the `speaker` parameter in the `base_speaker_tts.tts` method. Available choices: friendly, cheerful, excited, sad, angry, terrified, shouting, whispering. Note that the tone color embedding need to be updated. The speed can be controlled by the `speed` parameter. Let's try whispering with speed 0.9.

**Try with different languages.** OpenVoice can achieve multi-lingual voice cloning by simply replace the base speaker. We provide an example with a Chinese base speaker here and we encourage the readers to try `demo_part2.ipynb` for a detailed demo.

**Tech for good.** For people who will deploy OpenVoice for public usage: We offer you the option to add watermark to avoid potential misuse. Please see the ToneColorConverter class. **MyShell reserves the ability to detect whether an audio is generated by OpenVoice**, no matter whether the watermark is added or not.